# The First Solution
    This notebook analyses various outcomes on analyzing Amazon Reviews through Topic Modelling. Also, it proides insights on what value can be mined from this information.

### Reading the Data
     This piece of code unzips the package and parses the json document as a pandas dataframe.

In [10]:
data_dir = 'C:/Users/100048966/Documents/Citizen Data Science_C4/Code/Topic Modeling/'

In [3]:
import json
import os
import glob
import numpy as np
import random

In [29]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [11]:
df_reviews = getDF(data_dir+'reviews_Cell_Phones_and_Accessories_5.json.gz')
df_metadata = getDF(data_dir+'samsung_metadata.json.gz')

## Selecting the Item
    We need to be able to select the item to analyse. Through this flow the item and further details about it can be viewed.

In [15]:
df_reviews.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [16]:
df_metadata.head()

,asin,related,title,price,salesRank,imUrl,brand,categories,description
0,1466736038,"{'also_viewed': ['B008OK8IIY', 'B008MC3N34', '...",Samsung Galaxy S III I747 New Unlocked 16GB - ...,284.49,{'Cell Phones & Accessories': 56138},http://ecx.images-amazon.com/images/I/51hAD8gs...,,"[[Cell Phones & Accessories, Cell Phones, Unlo...","Brand New Unlocked Camera: 8 MP, 3264x2448 pix..."
1,1621911888,"{'also_bought': ['B007N9L140', 'B007C0CRDO', '...",Samsung Galaxy S Advance I9070 8Gb White WiFi ...,239.99,{'Cell Phones & Accessories': 209436},http://ecx.images-amazon.com/images/I/41oafU78...,NaN,"[[Cell Phones & Accessories, Cell Phones, Unlo...",The Samsung Galaxy S Advance I9070 offers its ...
2,9043435856,"{'also_viewed': ['B00B4HO91O', 'B000RW7IBK', '...",HTC Dash / S620 / S621 (Excalibur) Black Windo...,68.49,{'Cell Phones & Accessories': 89187},http://ecx.images-amazon.com/images/I/41%2B2y5...,NaN,"[[Cell Phones & Accessories, Cell Phones, Unlo...","Exclusive to T-Mobile, the quad-band T-Mobile ..."
3,B00006LIQC,NaN,Motorola V66 Phone (T-Mobile),NaN,{'Cell Phones & Accessories': 1196150},http://ecx.images-amazon.com/images/I/41BNBQHN...,NaN,"[[Cell Phones & Accessories, Cell Phones, Cont...",OverviewCompatible with T-Mobile cell phone se...
4,B00007E9TR,NaN,PCS Phone Samsung I330 (Sprint),NaN,{'Cell Phones & Accessories': 2225275},http://ecx.images-amazon.com/images/I/412JS9GK...,NaN,"[[Cell Phones & Accessories, Cell Phones, Cont...",The Samsung i330 is one of the standouts emerg...


In [26]:
df_merged = pd.merge(df_reviews, df_metadata, how='inner', on='asin' )
df_merged.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,related,title,price,salesRank,imUrl,brand,categories,description
0,AHGGSY182427,B0000SX3BK,Dave English,"[8, 10]","I recently moved to Phoenix, Arizona, and thou...",1.0,Nextel is a mess,1092268800,"08 12, 2004","{'also_viewed': ['B0041LR5TS', 'B0011TSQQ4', '...",Motorola i730 Phone (Nextel),NaN,{'Cell Phones & Accessories': 185224},http://ecx.images-amazon.com/images/I/31G6XSG4...,NaN,"[[Cell Phones & Accessories, Cell Phones, Unlo...",NaN
1,A1KD8NJPZ01R37,B0000SX3BK,doppelganger,"[2, 6]",And it was pretty much worth it (if you sign t...,5.0,swapped an i95cl for the .06 slimmer chassis o...,1070668800,"12 6, 2003","{'also_viewed': ['B0041LR5TS', 'B0011TSQQ4', '...",Motorola i730 Phone (Nextel),NaN,{'Cell Phones & Accessories': 185224},http://ecx.images-amazon.com/images/I/31G6XSG4...,NaN,"[[Cell Phones & Accessories, Cell Phones, Unlo...",NaN
2,A2MLBUX9REGYUI,B0000SX3BK,Dtho,"[4, 5]",I've glanced at a few reviews and looked at th...,5.0,Exceptional Quality,1094342400,"09 5, 2004","{'also_viewed': ['B0041LR5TS', 'B0011TSQQ4', '...",Motorola i730 Phone (Nextel),NaN,{'Cell Phones & Accessories': 185224},http://ecx.images-amazon.com/images/I/31G6XSG4...,NaN,"[[Cell Phones & Accessories, Cell Phones, Unlo...",NaN
3,A5JLAU2ARJ0BO,B0000SX3BK,"Gadgester ""No Time, No Money""","[5, 5]",Other than the high initial cost (I got it bef...,5.0,very happy,1074729600,"01 22, 2004","{'also_viewed': ['B0041LR5TS', 'B0011TSQQ4', '...",Motorola i730 Phone (Nextel),NaN,{'Cell Phones & Accessories': 185224},http://ecx.images-amazon.com/images/I/31G6XSG4...,NaN,"[[Cell Phones & Accessories, Cell Phones, Unlo...",NaN
4,AGZ8HQO485IL8,B0000SX3BK,J. Decelles,"[3, 6]",I don't own this phone. I own the cheaper Moto...,3.0,"Not a bad phone, but the cheaper model gets be...",1113523200,"04 15, 2005","{'also_viewed': ['B0041LR5TS', 'B0011TSQQ4', '...",Motorola i730 Phone (Nextel),NaN,{'Cell Phones & Accessories': 185224},http://ecx.images-amazon.com/images/I/31G6XSG4...,NaN,"[[Cell Phones & Accessories, Cell Phones, Unlo...",NaN


We can be able to view the names of all items present in the file, hence a new data frame is created.

In [17]:
df_item_names = df_metadata.loc[:,['asin','title']]
df_item_names.head()

,asin,title
0,1466736038,Samsung Galaxy S III I747 New Unlocked 16GB - ...
1,1621911888,Samsung Galaxy S Advance I9070 8Gb White WiFi ...
2,9043435856,HTC Dash / S620 / S621 (Excalibur) Black Windo...
3,B00006LIQC,Motorola V66 Phone (T-Mobile)
4,B00007E9TR,PCS Phone Samsung I330 (Sprint)


In [22]:
df_item_names.loc[df_item_names.asin.isin(['1466736038']),['title']]

,title
0,Samsung Galaxy S III I747 New Unlocked 16GB - ...


In [19]:
def df_item(asin):
    '''This function intakes an item ID or asin and returns the data frame of all the reviews of that item.'''
    reviewed_item = df_reviews.loc[df_reviews.asin.isin([asin]),['reviewText']]
    return reviewed_item

In [28]:
df_item('B0000SX3BK')

,reviewText
528,"I recently moved to Phoenix, Arizona, and thou..."
529,And it was pretty much worth it (if you sign t...
530,I've glanced at a few reviews and looked at th...
531,Other than the high initial cost (I got it bef...
532,I don't own this phone. I own the cheaper Moto...
533,I have to say that this is a great phone and t...


### Functions to Extract Nouns
    By using the pos tagger in the NLTK we can extract the nouns from a given sentence/ review.

In [34]:
import nltk as nt
noun_tags = ["NN","NNP","NNS","POS","WP"] #"PRP$" , "PRP", removing proper nouns

In [35]:
def tags(sentence):
    '''Takes a sentence and returns an array of tags'''
    array_words = nt.word_tokenize(sentence)
    tags = nt.pos_tag(array_words)
    return tags

In [36]:
def noun_words(review):
    '''Takes an array of sentence and returns a numpy array of nouns for each sentence'''
    nouns =[];
    all_tuples = tags(review)
    for one in all_tuples:
        for each in noun_tags:
            if(each == one[1]):
                nouns.append(one[0])
    return nouns

In [37]:
def nouns_only(corpus):
    '''This takes an array of sentences and returns a numpy array of nouns for each sentence'''
    total_nouns = []
    for each in corpus:
        total_nouns.append(noun_words(each))
    return total_nouns

## Creating Lda Model using GENSIM

This piece of code creates a document term matrix that can further be used to build LDA models.

Testing for complete reviews on the same item.

In [38]:
import gensim
from gensim import corpora

def lda_model(noun_array,n,k):
    '''This takes in a numpy array of nounsnumber of topics and number of words in each topic and returns the ldamodel. 
        Number of passes is hardcoded to 50'''
    # Creating the term dictionary of our corpus, where every unique term is assigned an index. 
    dictionary = corpora.Dictionary(noun_array)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(each) for each in noun_array]
    # Creating the object for LDA model using gensim library
    Lda = gensim.models.ldamodel.LdaModel
    # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=n, id2word = dictionary, passes=50)
    
    return ldamodel

C:\Users\100048966\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Forming a new data frame for the reviews of a selected items

In [30]:
'''Creating a dataframe of items with required item reviews'''
reviewed_item = df_item('B0000SX3BK')
final_corp = reviewed_item['reviewText']

In [31]:
final_corp.head()

528    I recently moved to Phoenix, Arizona, and thou...
529    And it was pretty much worth it (if you sign t...
530    I've glanced at a few reviews and looked at th...
531    Other than the high initial cost (I got it bef...
532    I don't own this phone. I own the cheaper Moto...
Name: reviewText, dtype: object

## Extracting only Nouns and various forms of nouns
Nouns were extracted, and proper nouns are not considered (I,we , them, they, etc)

In [39]:
total_nouns = nouns_only(final_corp)
len(total_nouns) #This gives the frequency of the reviews

6

In [40]:
first_model = lda_model(total_nouns,5,3)

In [41]:
a =first_model.print_topics(num_topics=5, num_words=3)

In [42]:
a

[(0, '0.068*"phone" + 0.048*"i730" + 0.048*"i710"'),
 (1, '0.101*"phone" + 0.026*"software" + 0.018*"battery"'),
 (2, '0.041*"Nextel" + 0.041*"phone" + 0.022*"Verizon"'),
 (3, '0.006*"features" + 0.006*"plan" + 0.006*"pricey"'),
 (4, '0.051*"Nextel" + 0.051*"phone" + 0.051*"number"')]

### Documenting the results for knowing the optimum number of reviews:
    Now we would run the topic modelling on different number of reviews for the same item and define the optimum percentage of reviews that needs to be sent to obtain the best accurate results.

In [43]:
def results_array(asin,k,l):
    '''Takes arguments: ASIN, number of topics, number of words in each topic.
    returns an array of number of reviews ans corresponding topics for each iteration.'''
    results = [[],[]]
    reviewed_item = df_item(asin)
    final_corp = reviewed_item['reviewText']
    print(final_corp.head())
    total_nouns = nouns_only(final_corp)
    print('\ntotal reviews for this item : ',len(total_nouns))
    
    percent = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
    val = [round(i * len(total_nouns)) for i in percent]

    for each in val:
        a = None
        model = lda_model(total_nouns[:each],k,l)
        a = model.print_topics(num_topics=k, num_words=l)
        results[0].append(each),results[1].append(a)
    
    return results

In [44]:
help(results_array)

Help on function results_array in module __main__:

results_array(asin, k, l)
    Takes arguments: ASIN, number of topics, number of words in each topic.
    returns an array of number of reviews ans corresponding topics for each iteration.



In [46]:
results_array('B0000SX3BK',4,2) #Most generalized

528    I recently moved to Phoenix, Arizona, and thou...
529    And it was pretty much worth it (if you sign t...
530    I've glanced at a few reviews and looked at th...
531    Other than the high initial cost (I got it bef...
532    I don't own this phone. I own the cheaper Moto...
Name: reviewText, dtype: object

total reviews for this item :  6


[[1, 1, 2, 2, 3, 4, 4, 5, 5, 6],
 [[(0, '0.022*"fees" + 0.022*"way"'),
   (1, '0.022*"number" + 0.022*"phone"'),
   (2, '0.064*"Nextel" + 0.064*"phone"'),
   (3, '0.022*"Alto" + 0.022*"taxes"')],
  [(0, '0.022*"Arizona" + 0.022*"service.If"'),
   (1, '0.022*"CA" + 0.022*"Cool"'),
   (2, '0.064*"number" + 0.064*"phone"'),
   (3, '0.022*"time" + 0.022*"closest"')],
  [(0, '0.063*"phone" + 0.063*"number"'),
   (1, '0.091*"plan" + 0.051*"course"'),
   (2, '0.018*"phone" + 0.018*"features"'),
   (3, '0.018*"phone" + 0.018*"picture"')],
  [(0, '0.091*"plan" + 0.051*"i95"'),
   (1, '0.018*"phone" + 0.018*"GPS"'),
   (2, '0.063*"phone" + 0.063*"number"'),
   (3, '0.018*"phone" + 0.018*"gain"')],
  [(0, '0.110*"phone" + 0.032*"software"'),
   (1, '0.010*"GPS" + 0.010*"pricey"'),
   (2, '0.059*"phone" + 0.049*"Nextel"'),
   (3, '0.010*"GPS" + 0.010*"features"')],
  [(0, '0.104*"phone" + 0.026*"software"'),
   (1, '0.009*"GPS" + 0.009*"pricey"'),
   (2, '0.057*"phone" + 0.048*"Nextel"'),
   (3, '

In [47]:
results_array('B0000SX3BK',3,3)  #moderate results

528    I recently moved to Phoenix, Arizona, and thou...
529    And it was pretty much worth it (if you sign t...
530    I've glanced at a few reviews and looked at th...
531    Other than the high initial cost (I got it bef...
532    I don't own this phone. I own the cheaper Moto...
Name: reviewText, dtype: object

total reviews for this item :  6


[[1, 1, 2, 2, 3, 4, 4, 5, 5, 6],
 [[(0, '0.022*"state" + 0.022*"connect" + 0.022*"Cool"'),
   (1, '0.063*"Nextel" + 0.063*"phone" + 0.063*"number"'),
   (2, '0.022*"Cool" + 0.022*"time" + 0.022*"numbers"')],
  [(0, '0.063*"number" + 0.063*"phone" + 0.063*"Nextel"'),
   (1, '0.022*"numbers" + 0.022*"miles" + 0.022*"function"'),
   (2, '0.022*"call" + 0.022*"who" + 0.022*"stars"')],
  [(0, '0.018*"phone" + 0.018*"gain" + 0.018*"picture"'),
   (1, '0.060*"phone" + 0.060*"number" + 0.060*"Nextel"'),
   (2, '0.080*"plan" + 0.045*"i95" + 0.045*"GPS"')],
  [(0, '0.080*"plan" + 0.045*"i95" + 0.045*"features"'),
   (1, '0.018*"phone" + 0.018*"GPS" + 0.018*"course"'),
   (2, '0.060*"phone" + 0.060*"Nextel" + 0.060*"number"')],
  [(0, '0.101*"phone" + 0.027*"software" + 0.019*"screen"'),
   (1, '0.010*"remorse" + 0.010*"pricey" + 0.010*"features"'),
   (2, '0.051*"number" + 0.051*"Nextel" + 0.051*"phone"')],
  [(0, '0.009*"remorse" + 0.009*"i95" + 0.009*"features"'),
   (1, '0.097*"phone" + 0.023

In [52]:
results_array('B0000SX3BK',4,3) #70%

528    I recently moved to Phoenix, Arizona, and thou...
529    And it was pretty much worth it (if you sign t...
530    I've glanced at a few reviews and looked at th...
531    Other than the high initial cost (I got it bef...
532    I don't own this phone. I own the cheaper Moto...
Name: reviewText, dtype: object

total reviews for this item :  6


[[1, 1, 2, 2, 3, 4, 4, 5, 5, 6],
 [[(0, '0.064*"Nextel" + 0.064*"number" + 0.064*"phone"'),
   (1, '0.022*"stars" + 0.022*"who" + 0.022*"Arizona"'),
   (2, '0.022*"fees" + 0.022*"Nothing" + 0.022*"past"'),
   (3, '0.022*"miles" + 0.022*"service.If" + 0.022*"assessments"')],
  [(0, '0.064*"number" + 0.064*"phone" + 0.064*"Nextel"'),
   (1, '0.022*"couple" + 0.022*"hour" + 0.022*"way"'),
   (2, '0.022*"numbers" + 0.022*"Palo" + 0.022*"couple"'),
   (3, '0.022*"Nextel" + 0.022*"phone" + 0.022*"number"')],
  [(0, '0.018*"phone" + 0.018*"i95" + 0.018*"picture"'),
   (1, '0.063*"phone" + 0.063*"number" + 0.063*"Nextel"'),
   (2, '0.091*"plan" + 0.051*"GPS" + 0.051*"course"'),
   (3, '0.018*"phone" + 0.018*"features" + 0.018*"i95"')],
  [(0, '0.018*"phone" + 0.018*"pricey" + 0.018*"picture"'),
   (1, '0.091*"plan" + 0.051*"course" + 0.051*"picture"'),
   (2, '0.018*"phone" + 0.018*"i95" + 0.018*"gain"'),
   (3, '0.063*"phone" + 0.063*"Nextel" + 0.063*"number"')],
  [(0, '0.055*"phone" + 0.055

In [53]:
results_array('B0000SX3BK',4,4) #70%

528    I recently moved to Phoenix, Arizona, and thou...
529    And it was pretty much worth it (if you sign t...
530    I've glanced at a few reviews and looked at th...
531    Other than the high initial cost (I got it bef...
532    I don't own this phone. I own the cheaper Moto...
Name: reviewText, dtype: object

total reviews for this item :  6


[[1, 1, 2, 2, 3, 4, 4, 5, 5, 6],
 [[(0, '0.022*"numbers" + 0.022*"couple" + 0.022*"stars" + 0.022*"friend"'),
   (1, '0.022*"anything" + 0.022*"return" + 0.022*"state" + 0.022*"stars"'),
   (2,
    '0.022*"miles" + 0.022*"connect" + 0.022*"service.If" + 0.022*"Arizona"'),
   (3, '0.064*"number" + 0.064*"Nextel" + 0.064*"phone" + 0.052*"quot"')],
  [(0, '0.064*"Nextel" + 0.064*"number" + 0.064*"phone" + 0.052*"quot"'),
   (1, '0.022*"i730" + 0.022*"friend" + 0.022*"Palo" + 0.022*"function"'),
   (2, '0.022*"way" + 0.022*"state" + 0.022*"Alto" + 0.022*"friend"'),
   (3, '0.022*"way" + 0.022*"closest" + 0.022*"Alto" + 0.022*"drive"')],
  [(0, '0.091*"plan" + 0.051*"GPS" + 0.051*"remorse" + 0.051*"gain"'),
   (1, '0.018*"phone" + 0.018*"gain" + 0.018*"pricey" + 0.018*"course"'),
   (2, '0.063*"phone" + 0.063*"number" + 0.063*"Nextel" + 0.051*"quot"'),
   (3, '0.018*"phone" + 0.018*"pricey" + 0.018*"course" + 0.018*"gain"')],
  [(0, '0.091*"plan" + 0.051*"pricey" + 0.051*"remorse" + 0.051*"

In [54]:
results_array('B0000SX3BK',4,5) #Not useful at all

528    I recently moved to Phoenix, Arizona, and thou...
529    And it was pretty much worth it (if you sign t...
530    I've glanced at a few reviews and looked at th...
531    Other than the high initial cost (I got it bef...
532    I don't own this phone. I own the cheaper Moto...
Name: reviewText, dtype: object

total reviews for this item :  6


[[1, 1, 2, 2, 3, 4, 4, 5, 5, 6],
 [[(0,
    '0.064*"Nextel" + 0.064*"phone" + 0.064*"number" + 0.052*"quot" + 0.040*"Phoenix"'),
   (1,
    '0.022*"CA" + 0.022*"Palo" + 0.022*"miles" + 0.022*"drive" + 0.022*"time"'),
   (2,
    '0.022*"anything" + 0.022*"CA" + 0.022*"calls" + 0.022*"call" + 0.022*"Palo"'),
   (3,
    '0.022*"CA" + 0.022*"i730" + 0.022*"fees" + 0.022*"Palo" + 0.022*"anything"')],
  [(0,
    '0.022*"day" + 0.022*"function" + 0.022*"color" + 0.022*"assessments" + 0.022*"Alto"'),
   (1,
    '0.064*"Nextel" + 0.064*"number" + 0.064*"phone" + 0.052*"quot" + 0.040*"Phoenix"'),
   (2,
    '0.022*"closest" + 0.022*"CA" + 0.022*"Arizona" + 0.022*"miles" + 0.022*"stars"'),
   (3,
    '0.022*"phone" + 0.022*"number" + 0.022*"Nextel" + 0.022*"calls" + 0.022*"time"')],
  [(0,
    '0.018*"phone" + 0.018*"pricey" + 0.018*"course" + 0.018*"features" + 0.018*"gain"'),
   (1,
    '0.063*"phone" + 0.063*"number" + 0.063*"Nextel" + 0.051*"quot" + 0.039*"Phoenix"'),
   (2,
    '0.018*"phone

In [55]:
results_array('B0000SX3BK',5,3) #At 80% of total reviews we can see req topics with least probabilities.

528    I recently moved to Phoenix, Arizona, and thou...
529    And it was pretty much worth it (if you sign t...
530    I've glanced at a few reviews and looked at th...
531    Other than the high initial cost (I got it bef...
532    I don't own this phone. I own the cheaper Moto...
Name: reviewText, dtype: object

total reviews for this item :  6


[[1, 1, 2, 2, 3, 4, 4, 5, 5, 6],
 [[(0, '0.022*"who" + 0.022*"closest" + 0.022*"CA"'),
   (1, '0.022*"Nextel" + 0.022*"number" + 0.022*"quot"'),
   (2, '0.022*"drive" + 0.022*"closest" + 0.022*"function"'),
   (3, '0.022*"Phoenix" + 0.022*"i730" + 0.022*"past"'),
   (4, '0.066*"phone" + 0.066*"number" + 0.066*"Nextel"')],
  [(0, '0.022*"service.If" + 0.022*"time" + 0.022*"way"'),
   (1, '0.022*"time" + 0.022*"who" + 0.022*"Palo"'),
   (2, '0.066*"number" + 0.066*"phone" + 0.066*"Nextel"'),
   (3, '0.022*"past" + 0.022*"service.If" + 0.022*"Palo"'),
   (4, '0.022*"phone" + 0.022*"number" + 0.022*"Nextel"')],
  [(0, '0.018*"phone" + 0.018*"remorse" + 0.018*"picture"'),
   (1, '0.018*"phone" + 0.018*"remorse" + 0.018*"picture"'),
   (2, '0.064*"phone" + 0.064*"Nextel" + 0.064*"number"'),
   (3, '0.100*"plan" + 0.055*"i95" + 0.055*"pricey"'),
   (4, '0.018*"phone" + 0.018*"course" + 0.018*"remorse"')],
  [(0, '0.018*"phone" + 0.018*"features" + 0.018*"i95"'),
   (1, '0.100*"plan" + 0.055*"

# Observations:
    It is observed that at 70% of total reviews, with 4 topics and 4 words each, we have accurate results.

### Working with a different item

In [ ]:
#Should run the code with different item and different data. at 70% reviews, 4 and 4 topics and words..